# 🦎 Youtube Video Downloader 🦎

## 🐍 Encubadora 🐍

In [ ]:
# import pytube

# url = 'https://www.youtube.com/watch?v=eycU0vO9Gzc'

# yt = pytube.YouTube(url)

# video = yt.streams.first()

# video.download(r"C:\Users\matheus_meissner\Desktop\github\yt_video_downloader")

## 🐢 Depuração 🐢

### Bibliotecas

In [ ]:
import pytube

### Escolhe a qualidade 💠

In [ ]:
def escolher_qualidade(url):
    yt = pytube.YouTube(url)
    streams = yt.streams.filter(progressive=True).all()
    
    print("Qualidade disponível:")
    for i, stream in enumerate(streams):
        print(f"{i+1}. {stream.resolution} - {stream.mime_type}")
    
    escolha = int(input("Escolha o número da qualidade desejada: "))
    selected_stream = streams[escolha-1]
    
    return selected_stream

### Baixa o vídeo

In [ ]:
def download_video(url, output_path):
    stream = escolher_qualidade(url)
    stream.download(output_path)
    print("Download concluído.")

### Exemplo de uso 💠

In [ ]:
# Exemplo de uso:
url = input("https://www.youtube.com/watch?v=nNkw3Fo9Aqk")
output_path = input(r"C:\Users\matheus_meissner\Desktop\github\yt_video_downloader")

download_video(url, output_path)

## Teste GPT

In [ ]:
import tkinter as tk
from tkinter import messagebox, filedialog
from pytube import YouTube
import os
import threading

def select_download_path():
    download_path = filedialog.askdirectory()
    if download_path:
        download_path_entry.delete(0, tk.END)
        download_path_entry.insert(0, download_path)

def update_quality_dropdown():
    try:
        qualities = get_available_qualities()
        quality_var.set("")  # Limpar o valor atual do menu suspenso
        quality_dropdown["menu"].delete(0, "end")  # Limpar as opções do menu suspenso
        for quality in qualities:
            quality_dropdown["menu"].add_command(label=quality, command=tk._setit(quality_var, quality))
    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao obter as qualidades disponíveis: {str(e)}")
        return

def update_format_dropdown():
    try:
        formats = get_available_formats()
        format_var.set("")  # Limpar o valor atual do menu suspenso
        format_dropdown["menu"].delete(0, "end")  # Limpar as opções do menu suspenso
        for format in formats:
            format_dropdown["menu"].add_command(label=format, command=tk._setit(format_var, format))
    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao obter os formatos disponíveis: {str(e)}")
        return

def get_available_qualities():
    url = url_entry.get()
    try:
        yt = YouTube(url)
        # Filtrar as qualidades disponíveis para evitar duplicatas e remover opções indesejadas
        available_qualities = set()
        for stream in yt.streams.filter(type="video"):
            if stream.resolution not in available_qualities and stream.resolution is not None:
                available_qualities.add(stream.resolution)
        return sorted(available_qualities)
    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao obter as qualidades disponíveis: {str(e)}")
        return []

def get_available_formats():
    url = url_entry.get()
    try:
        yt = YouTube(url)
        # Obter os formatos disponíveis para o vídeo
        return sorted(set(stream.mime_type.split("/")[1] for stream in yt.streams))
    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao obter os formatos disponíveis: {str(e)}")
        return []

def download_video():
    url = url_entry.get()
    selected_quality = quality_var.get()
    selected_format = format_var.get()
    try:
        yt = YouTube(url)
        # Verificar se a opção de qualidade e formato selecionados são suportados
        video = yt.streams.filter(type="video", resolution=selected_quality, mime_type=f"video/{selected_format}").first()
        if video is None:
            messagebox.showerror("Erro", f"A combinação de qualidade '{selected_quality}' e formato '{selected_format}' não é suportada para este vídeo.")
            return
        download_path = download_path_entry.get()
        file_path = os.path.join(download_path, video.default_filename)
        video.download(download_path)
        
        # Converter para MP4 se o formato não for MP4
        if selected_format != "mp4":
            converted_file_path = os.path.splitext(file_path)[0] + ".mp4"
            os.rename(file_path, converted_file_path)
            messagebox.showinfo("Sucesso", f"Vídeo baixado e convertido para MP4 com sucesso! Salvo em: {converted_file_path}")
        else:
            messagebox.showinfo("Sucesso", f"Vídeo baixado com sucesso! Salvo em: {file_path}")
    except Exception as e:
        messagebox.showerror("Erro", f"Ocorreu um erro ao baixar o vídeo: {str(e)}")

# Criar a janela principal
root = tk.Tk()
root.title("YouTube Video Downloader")

# Criar os widgets
url_label = tk.Label(root, text="URL do vídeo:")
url_label.grid(row=0, column=0, padx=10, pady=5, sticky=tk.W)

url_entry = tk.Entry(root, width=50)
url_entry.grid(row=0, column=1, padx=10, pady=5)

download_path_label = tk.Label(root, text="Caminho de Download:")
download_path_label.grid(row=1, column=0, padx=10, pady=5, sticky=tk.W)

download_path_entry = tk.Entry(root, width=50)
download_path_entry.grid(row=1, column=1, padx=10, pady=5)

browse_button = tk.Button(root, text="Navegar...", command=select_download_path)
browse_button.grid(row=1, column=2, padx=5, pady=5)

quality_label = tk.Label(root, text="Qualidade do vídeo:")
quality_label.grid(row=2, column=0, padx=10, pady=5, sticky=tk.W)

quality_var = tk.StringVar(root)
quality_dropdown = tk.OptionMenu(root, quality_var, "")
quality_dropdown.grid(row=2, column=1, padx=10, pady=5)

format_label = tk.Label(root, text="Formato do vídeo:")
format_label.grid(row=3, column=0, padx=10, pady=5, sticky=tk.W)

format_var = tk.StringVar(root)
format_dropdown = tk.OptionMenu(root, format_var, "")
format_dropdown.grid(row=3, column=1, padx=10, pady=5)

download_button = tk.Button(root, text="Baixar Vídeo", command=download_video)
download_button.grid(row=4, column=0, columnspan=2, pady=10)

# Atualizar as opções de qualidade e formato quando o usuário inserir o link do vídeo
def on_url_entry_change(event):
    update_quality_dropdown()
    update_format_dropdown()
    thread = threading.Thread(target=fetch_quality_and_format_info)
    thread.start()

def fetch_quality_and_format_info():
    update_quality_dropdown()
    update_format_dropdown()

url_entry.bind("<KeyRelease>", on_url_entry_change)

# Executar a aplicação
root.mainloop()